# Aula 02 - Leitura e recuperação de dados de diversas fontes

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [ ]:
!pip install lxml
!pip install xlrd
!pip install openpyxl
!pip install requests
!pip install sqlalchemy

## Leitura e escrita dos dados no formato texto

In [ ]:
!cat files/ex1.csv

In [ ]:
df = pd.read_csv('files/ex1.csv')
df

In [ ]:
pd.read_table('files/ex1.csv', sep=',')

In [ ]:
!cat files/ex2.csv

In [ ]:
pd.read_csv('files/ex2.csv', header=None)

In [ ]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('files/ex2.csv', names=names, index_col='message')

In [ ]:
!cat files/csv_mindex.csv
parsed = pd.read_csv('files/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

In [ ]:
list(open('files/ex3.txt'))

In [ ]:
result = pd.read_table('files/ex3.txt', sep='\s+')
result

In [ ]:
!cat files/ex4.csv
pd.read_csv('files/ex4.csv', skiprows=[0, 2, 3])

In [ ]:
!cat files/ex5.csv
result = pd.read_csv('files/ex5.csv')
print()
print(result)
print(pd.isnull(result))

In [ ]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('files/ex5.csv', na_values=sentinels)

### Leitura de arquivos texto por partes

In [ ]:
pd.options.display.max_rows = 10

In [ ]:
result = pd.read_csv('files/ex6.csv')
result

In [ ]:
pd.read_csv('files/ex6.csv', nrows=5)

In [ ]:
chunker = pd.read_csv('files/ex6.csv', chunksize=1000)
chunker

In [ ]:
chunker = pd.read_csv('files/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [ ]:
tot[:10]

### Escrevendo dados no formato texto

In [ ]:
data = pd.read_csv('files/ex5.csv')
data

In [ ]:
data.to_csv('files/out.csv')
!cat files/out.csv

In [ ]:
import sys
data.to_csv(sys.stdout, sep='|')

In [ ]:
data.to_csv(sys.stdout, na_rep='NULL')

In [ ]:
data.to_csv(sys.stdout, index=False, header=False)

In [ ]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

### Documentos JSON

In [ ]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [ ]:
import json
result = json.loads(obj)
result

In [ ]:
asjson = json.dumps(result)

In [ ]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

In [ ]:
!cat files/example.json

In [ ]:
data = pd.read_json('files/example.json')
data

### XML e HTML: Web Scraping

In [ ]:
tables = pd.read_html('files/fdic_failed_bank_list.html')
len(tables)
failures = tables[0]
failures.head()

In [ ]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

#### Parsing XML com lxml.objectify

In [2]:
from lxml import objectify

path = 'files/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [4]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [5]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8


In [6]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [7]:
root
root.get('href')
root.text

'Google'

## Leitura de formatos binários

In [8]:
frame = pd.read_csv('files/ex1.csv')
frame
frame.to_pickle('files/frame_pickle')

In [9]:
pd.read_pickle('files/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
!rm files/frame_pickle

### Lendo arquivos do Excel

In [11]:
xlsx = pd.ExcelFile('files/ex1.xlsx')

In [12]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [13]:
frame = pd.read_excel('files/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [14]:
writer = pd.ExcelWriter('files/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [16]:
frame.to_excel('files/ex2.xlsx')

In [17]:
!rm files/ex2.xlsx

## Interagindo com Web APIs

In [18]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [20]:
data = resp.json()
data[0]['title']

'CLN: Break up wrap applied output'

In [21]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,36536,CLN: Break up wrap applied output,"[{'id': 697792067, 'node_id': 'MDU6TGFiZWw2OTc...",open
1,36535,Regr/period range large value/issue 36430,"[{'id': 60635328, 'node_id': 'MDU6TGFiZWw2MDYz...",open
2,36534,CLN: clean up blocks.py,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
3,36531,Add generate pip dependency's from conda to pr...,"[{'id': 219960758, 'node_id': 'MDU6TGFiZWwyMTk...",open
4,36530,REF: Categorical.fillna match patterns in othe...,"[{'id': 1741841389, 'node_id': 'MDU6TGFiZWwxNz...",open
5,36529,add generate_pip_deps_from_conda.py to pre-commit,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
6,36528,REF: test_to_latex,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
7,36526,BUG: modulo of pd.Int64Index returns negative ...,"[{'id': 47223669, 'node_id': 'MDU6TGFiZWw0NzIy...",open
8,36525,BUG:,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
9,36523,DOC: a few sphinx fixes in release notes,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open


## Interagindo com Bancos de dados

In [22]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [23]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [24]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [25]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [26]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [27]:
!rm mydata.sqlite